In [33]:
import pandas as pd

In [34]:
df= pd.read_csv('/content/DailyDetailedPerformanceV0831.csv')

In [35]:
df.payment_type.unique()

array(['Coin', 'Apple-Pay', 'Stored-Value', 'CreditCard-SR', 'Google-Pay',
       'Digital-Wallet', 'bill'], dtype=object)

In [43]:
pcts=[.05,.1,.2,.25,.5,.75,.9,.95,.96,.97,.98,.99,.995,.996,.997,.998,.999]
pctls=['5%', '10%', '20%', '25%', '50%', '75%', '90%', '95%', '96%', '97%', '98%', '99%', '99.5%', '99.6%', '99.7%', '99.8%', '99.9%']
pctl_series=df.no_of_vends.describe(percentiles=pcts)
df['AppUser']= df.loc[df.payment_type=='Stored-Value', 'transactor_id']
df['AppSpend']= df.loc[df.payment_type=='Stored-Value','collected_amount']
df['DigitalUser']= df.loc[df.payment_type!='Coin', 'transactor_id']
df['DigitalSpend']= df.loc[df.payment_type!='Coin','collected_amount']
df['DigitalVend']= df.payment_type!='Coin' 
met_dict= {'installcap': 'min', 
           'no_of_vends': 'sum', 
           'machine_id':'nunique', 
           'collected_amount': 'sum', 
           'transactor_id': 'nunique',
           'AppUser': 'nunique', 
           'AppSpend': 'sum', 
           'DigitalUser': 'nunique', 
           'DigitalSpend': 'sum',
           'DigitalVend': 'sum'}

In [44]:
v= pctl_series[pctls[-2]]
df1= df.loc[df.no_of_vends <= v]

In [50]:
df2= df1.groupby(['cohort_flag', 'location_id']).agg(met_dict).reset_index()
df2.columns=['cohort_flag', 'location_id','InstalledMachines', 'Vends', 'TxnMachines', 'SpendAmount', "AllUser", 
             'AppUser', 'AppSpendAmount', 'DigitalUser','DigitalSpend', 'DigitalVend']

In [53]:
df2.head()

,cohort_flag,location_id,InstalledMachines,Vends,TxnMachines,SpendAmount,AllUser,AppUser,AppSpendAmount,DigitalUser,DigitalSpend,DigitalVend,UsedMachinePct,ARPAU,GRPM,VendsDigitalUser,SpendDigitalUser
0,control,001c980e-9557-4f85-8336-7653d85a618b,15,194,10,21875,1,1,650.0,1,650.0,5,0.666667,6.500,21.875000,5.000000,650.000000
1,control,01272c71-39ba-4d97-81a0-eb6e1fe00d4c,5,2,2,375,1,1,175.0,1,175.0,1,0.400000,1.750,1.875000,1.000000,175.000000
2,control,01ce7901-de7f-4ba8-835c-4a852455678e,11,66,9,10700,14,4,1950.0,14,7800.0,46,0.818182,4.875,11.888889,3.285714,557.142857
3,control,026926d1-e09a-42b1-b231-ff264495a10a,2,6,2,650,2,0,0.0,2,650.0,4,1.000000,NaN,3.250000,2.000000,325.000000
4,control,02e08450-d3f5-4e57-8487-96ec9cc64546,4,15,3,2250,0,0,0.0,0,0.0,0,0.750000,NaN,7.500000,NaN,NaN


In [79]:
df2['UsedMachinePct']= df2.TxnMachines/df2.InstalledMachines
df2['ARPAU']= df2.AppSpendAmount/(100.00*df2.AppUser)
df2['GRPM']= df2.SpendAmount/(100.0*df2.TxnMachines)
df2['VendsDigitalUser']= df2.DigitalVend/df2.DigitalUser
df2['SpendDigitalUser']= df2.DigitalSpend/ (100.0*df2.DigitalUser)

In [80]:
df3= df2.groupby('cohort_flag').agg({'location_id': 'nunique','InstalledMachines': 'sum','Vends': 'sum',
       'TxnMachines': 'sum', 'SpendAmount': 'sum', 'AllUser': 'sum', 'AppUser': 'sum', 'AppSpendAmount': 'sum',
       'DigitalUser': 'sum', 'DigitalSpend': 'sum', 'DigitalVend': 'sum' })

In [81]:
df2.head()

,cohort_flag,location_id,InstalledMachines,Vends,TxnMachines,SpendAmount,AllUser,AppUser,AppSpendAmount,DigitalUser,DigitalSpend,DigitalVend,UsedMachinePct,ARPAU,GRPM,VendsDigitalUser,SpendDigitalUser
0,control,001c980e-9557-4f85-8336-7653d85a618b,15,194,10,21875,1,1,650.0,1,650.0,5,0.666667,6.500,21.875000,5.000000,6.500000
1,control,01272c71-39ba-4d97-81a0-eb6e1fe00d4c,5,2,2,375,1,1,175.0,1,175.0,1,0.400000,1.750,1.875000,1.000000,1.750000
2,control,01ce7901-de7f-4ba8-835c-4a852455678e,11,66,9,10700,14,4,1950.0,14,7800.0,46,0.818182,4.875,11.888889,3.285714,5.571429
3,control,026926d1-e09a-42b1-b231-ff264495a10a,2,6,2,650,2,0,0.0,2,650.0,4,1.000000,NaN,3.250000,2.000000,3.250000
4,control,02e08450-d3f5-4e57-8487-96ec9cc64546,4,15,3,2250,0,0,0.0,0,0.0,0,0.750000,NaN,7.500000,NaN,NaN


In [82]:
df3['UsedMachinePct']= df3.TxnMachines/df3.InstalledMachines
df3['ARPAU']= df3.AppSpendAmount/(100.00*df3.AppUser)
df3['GRPM']= df3.SpendAmount/(100.0*df3.TxnMachines)
df3['VendsDigitalUser']= df3.DigitalVend/df3.DigitalUser
df3['SpendDigitalUser']= df3.DigitalSpend/ (100.0*df3.DigitalUser)
df3['VendsLocation']= df3.Vends/df3.location_id
df3['SpendLocation']= df3.DigitalSpend/(100.00*df3.location_id)

In [90]:
df_mean= df3.loc[:, ['ARPAU', 'GRPM',
       'VendsDigitalUser', 'SpendDigitalUser', 'VendsLocation',
       'SpendLocation']].T

cohort_flag,control,test
ARPAU,14.200902,14.433839
GRPM,38.173090,41.435779
VendsDigitalUser,6.119561,6.379515
SpendDigitalUser,11.963317,12.491835
VendsLocation,207.752705,225.959350
SpendLocation,92.637125,113.941078


In [96]:
from scipy import stats
for c in ['ARPAU', 'GRPM', 'VendsDigitalUser', 'SpendDigitalUser', 'DigitalVend', 'DigitalSpend']:
  control= df2.loc[(df2.cohort_flag== 'control'), c]
  test= df2.loc[(df2.cohort_flag== 'test'), c]
  t_score = stats.ttest_ind_from_stats(test.mean(), test.std(), test.shape[0], \
                               control.mean(), control.std(), control.shape[0], \
                               equal_var=False)
  print(c, test.mean(), test.std(), control.mean(), control.std(),t_score)



ARPAU 13.120125402778847 9.16162273705943 13.714254903980938 10.630355178929287 Ttest_indResult(statistic=-1.3143662446891735, pvalue=0.18906610852572037)
GRPM 37.57000023106597 31.821648774649788 36.097299496655815 32.15147501837784 Ttest_indResult(statistic=1.0530983055281504, pvalue=0.292561781066832)
VendsDigitalUser inf nan 5.568468651933381 3.6778008340779937 Ttest_indResult(statistic=nan, pvalue=nan)
SpendDigitalUser inf nan 11.42951963373264 8.377566015593194 Ttest_indResult(statistic=nan, pvalue=nan)
DigitalVend 58.18911276069282 179.19688608696444 47.38639876352396 107.54314707804788 Ttest_indResult(statistic=1.9982266641918858, pvalue=0.04586307787416769)
DigitalSpend 11394.107811947684 33657.50530603499 9263.712519319939 19538.750292074703 Ttest_indResult(statistic=2.140603992657296, pvalue=0.03245245108999421)


In [ ]:
df2.loc[df2['UsedMachinePct']<=df2.UsedMachinePct.describe(percentiles=[.05,.1,.15,.25, .5,.75, .95])['10%']]

,cohort_flag,location_id,InstalledMachines,Vends,TxnMachines,SpendAmount,DigitalUser,AppUser,AppSpendAmount,UsedMachinePct
9,control,06843e7c-e73d-4c91-b5c8-74a793538dc4,46,1,1,25,0,0,0.0,0.021739
13,control,0719ddde-2727-400d-92ee-251306a0d8bb,64,346,15,48550,4,3,4000.0,0.234375
14,control,0743f6ab-de61-456e-972f-e1d0092eb5ec,36,574,8,56075,17,8,11050.0,0.222222
22,control,0aae2157-f369-426e-90c0-755f8565e5f4,64,21,7,3150,1,0,0.0,0.109375
24,control,0afef6a8-e558-4392-8418-3d081c284eea,343,88,26,13150,4,1,225.0,0.075802
...,...,...,...,...,...,...,...,...,...,...
3454,test,fd1d810f-6623-43ab-83fe-6fb26500e2c4,19,6,3,1200,2,1,400.0,0.157895
3460,test,fdf45964-7fb1-4cb6-b8d9-0f35e8c62663,162,977,24,131000,17,9,14900.0,0.148148
3471,test,ff7e4613-5c05-4a30-8f40-8220324a4cd3,100,750,20,121425,17,15,23125.0,0.200000
3473,test,ffbbd0db-bddb-4447-9a68-e5c975be3019,5,1,1,25,0,0,0.0,0.200000
